In [2]:
from statistics import mean
from tkinter import Y
import pandas as pd
import numpy as np
from scipy import interpolate
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from scipy.stats import iqr

In [3]:
data = pd.read_csv("Granite1.csv")
data = data.iloc[2:102]
data.index = np.arange(0, len(data))

X = data[[" X", " Y", " Hardness(GPa)", " Modulus"]]
tempdf = X.copy()
tempdf

,X,Y,Hardness(GPa),Modulus
0,-3.770873,0.002520,0.387642,6.648113
1,-2.771181,0.002699,1.224675,32.848149
2,-1.771469,0.002709,156.045745,1888.860122
3,-0.771161,0.002709,1.841265,37.080645
4,0.228789,0.002699,10.465644,178.110457
...,...,...,...,...
95,1.228045,-8.997162,25.739224,211.007222
96,2.227500,-8.997162,42.664982,681.374523
97,3.227748,-8.997162,63.865255,362.929557
98,4.227718,-8.997162,26.693745,185.871818


In [4]:
iqr_multiplier_hard = 1.5 * iqr(X[" Hardness(GPa)"])
third_quar_hard = np.percentile(X[" Hardness(GPa)"], 75)
first_quar_hard = np.percentile(X[" Hardness(GPa)"], 25)
iqr_multiplier_mod = 1.5 * iqr(X[" Modulus"])
third_quar_mod = np.percentile(X[" Modulus"], 75)
first_quar_mod = np.percentile(X[" Modulus"], 25)

for x in range(len(X[" Hardness(GPa)"])):
    if tempdf[" Hardness(GPa)"][x] > (iqr_multiplier_hard + third_quar_hard) or tempdf[" Hardness(GPa)"][x] < (first_quar_hard - iqr_multiplier_hard):
        tempdf[" Hardness(GPa)"][x] = None
    if tempdf[" Modulus"][x] > (iqr_multiplier_mod + third_quar_mod) or tempdf[" Modulus"][x] < (first_quar_mod - iqr_multiplier_mod):
        if tempdf[" Modulus"][x] > (iqr_multiplier_mod + third_quar_mod) or tempdf[" Modulus"][x] < (first_quar_mod - iqr_multiplier_mod):
            tempdf[" Modulus"][x] = None

tempdf

,X,Y,Hardness(GPa),Modulus
0,-3.770873,0.002520,0.387642,6.648113
1,-2.771181,0.002699,1.224675,32.848149
2,-1.771469,0.002709,NaN,NaN
3,-0.771161,0.002709,1.841265,37.080645
4,0.228789,0.002699,10.465644,178.110457
...,...,...,...,...
95,1.228045,-8.997162,25.739224,211.007222
96,2.227500,-8.997162,42.664982,NaN
97,3.227748,-8.997162,NaN,362.929557
98,4.227718,-8.997162,26.693745,185.871818


In [5]:
tempdf = tempdf.interpolate()
tempdf

,X,Y,Hardness(GPa),Modulus
0,-3.770873,0.002520,0.387642,6.648113
1,-2.771181,0.002699,1.224675,32.848149
2,-1.771469,0.002709,1.532970,34.964397
3,-0.771161,0.002709,1.841265,37.080645
4,0.228789,0.002699,10.465644,178.110457
...,...,...,...,...
95,1.228045,-8.997162,25.739224,211.007222
96,2.227500,-8.997162,42.664982,286.968390
97,3.227748,-8.997162,34.679363,362.929557
98,4.227718,-8.997162,26.693745,185.871818


In [6]:
max_hard = max(tempdf[" Hardness(GPa)"])
min_hard = min(tempdf[" Hardness(GPa)"])
max_mod =  max(tempdf[" Modulus"])
min_mod = min(tempdf[" Modulus"])
max_x = max(tempdf[" X"])
min_x = min(tempdf[" X"])
max_y = max(tempdf[" Y"])
min_y = min(tempdf[" Y"])

for x in range(len(tempdf)):
    tempdf[" Hardness(GPa)"][x] = (tempdf[" Hardness(GPa)"][x]-min_hard)/(max_hard-min_hard)
    tempdf[" Modulus"][x] = (tempdf[" Modulus"][x]-min_mod)/(max_mod-min_mod)
    tempdf[" Y"][x] = round((tempdf[" Y"][x]-min_y)/(max_y-min_y), 2)
    tempdf[" X"][x] = round((tempdf[" X"][x]-min_x)/(max_x-min_x), 2)

tempdf

,X,Y,Hardness(GPa),Modulus
0,0.00,1.0,0.000517,0.000000
1,0.11,1.0,0.017389,0.073537
2,0.22,1.0,0.023602,0.079477
3,0.33,1.0,0.029816,0.085417
4,0.44,1.0,0.203648,0.481255
...,...,...,...,...
95,0.56,0.0,0.511500,0.573589
96,0.67,0.0,0.852654,0.786794
97,0.78,0.0,0.691697,1.000000
98,0.89,0.0,0.530740,0.503040


In [7]:
tempdf = tempdf.sort_index()
tempdf

x = np.linspace(0, 1, 10)
y = np.linspace(1, 0, 10)
xx,yy = np.meshgrid(x,y)
z_h = tempdf[" Hardness(GPa)"].to_numpy().reshape([10,10])
z_m = tempdf[" Modulus"].to_numpy().reshape([10,10])

hard_interp_norm=interpolate.interp2d(x, y, z_h, kind="linear")
mod_interp_norm=interpolate.interp2d(x, y, z_m, kind="linear")

In [10]:
new_temp = pd.DataFrame()
pts = 20
feature_x = np.linspace(0, 1, pts)
feature_y = np.linspace(1, 0, pts)

new_temp["X"] = list(feature_x) * pts
arr = []
for i in list(feature_y):
    arr.append([i]*pts)
new_temp["Y"] = np.array(arr).reshape([pts*pts, 1])

feature_hard = hard_interp_norm(feature_x, feature_y).reshape([pts*pts,1])
feature_mod = mod_interp_norm(feature_x, feature_y).reshape([pts*pts,1])
feature_hard = [i[0] for i in feature_hard]
feature_mod = [i[0] for i in feature_mod]

#weights of x and y
new_temp["X"] = .1 * new_temp["X"]
new_temp["Y"] = .1 * new_temp["Y"]
new_temp["Hardness"] = feature_hard
new_temp["Modulus"] = feature_mod

new_temp
#new_temp.to_csv("G1.csv")

,X,Y,Hardness,Modulus
0,0.000000,0.1,0.101479,0.372610
1,0.005263,0.1,0.170869,0.334453
2,0.010526,0.1,0.240259,0.296296
3,0.015789,0.1,0.296054,0.358270
4,0.021053,0.1,0.350149,0.432760
...,...,...,...,...
395,0.078947,0.0,0.550582,0.528164
396,0.084211,0.0,0.405671,0.415974
397,0.089474,0.0,0.268498,0.314496
398,0.094737,0.0,0.193215,0.298703
